# SpotifyDB Notebook
This notebook demonstrates setting up and using the SpotifyDB utilities.

In [1]:
#in powershell
%pip install spotipy duckdb python-dotenv pandas


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## create cred.py  in the root directory 
add your own client id and secrets using the values you have optained from spotify.


### cred.py
```python
# get values from 
# https://developer.spotify.com/dashboard/
spotify_client_id = "your id"
spotify_client_secret = "your secret"
```


In [2]:
# import the credentials from cred.py
from cred import spotify_client_id, spotify_client_secret

## import libraries

In [3]:
import os
from dotenv import load_dotenv

import duckdb
import pandas as pd

import spotipy
from spotipy.oauth2 import SpotifyOAuth

# custom functions
import functions as fn


## setting up the environment, api, and duckdb

In [4]:
# Load .env
load_dotenv()

CLIENT_ID = spotify_client_id
CLIENT_SECRET = spotify_client_secret
REDIRECT_URI = "http://127.0.0.1:8000/callback"
SCOPE = (
    "user-library-read user-read-recently-played user-top-read user-read-playback-state user-follow-read playlist-read-private playlist-modify-private playlist-modify-public"
)

assert CLIENT_ID and CLIENT_SECRET and REDIRECT_URI, "Missing Spotify env vars"

sp = spotipy.Spotify(
    auth_manager=SpotifyOAuth(
        client_id=CLIENT_ID,
        client_secret=CLIENT_SECRET,
        redirect_uri=REDIRECT_URI,
        scope=SCOPE,
        open_browser=True,  # will open auth page in browser
        cache_path=".cache-spotifydb"  # token cache
    )
)

current_user = sp.current_user()
current_user["display_name"], current_user["id"]


('1257524228', '1257524228')

In [5]:
# In-memory DB (great for playing around)
# con = duckdb.connect(database=':memory:')

# If you want persistent on-disk:
con = duckdb.connect(database='spotify.duckdb')


In [6]:
df_tables = con.execute(f"SHOW TABLES").df()
display(df_tables)


,name
0,AI_Covers
1,Covers
2,NTS_Covers
3,followed_artist
4,my_liked_songs
5,table_updated


## function wrapper 

In [7]:
def get_item(table_name:str,source_type:str, source_id:str,verbose=True):
    """
    source_type: playlist, album, artist, liked_songs, top_tracks, recently_played
    """
    global con
    global sp

    table_age = fn.duckdb_table_age(con, table_name)    
    print(f"table age: {table_age} days old")

    if table_age is None or table_age > 1.0:

        if source_type == "playlist":
            items = fn.load_tracks_from_playlist(sp, source_id)
        elif source_type == "album":
            items = fn.load_tracks_from_album(sp, source_id)
        elif source_type == "artist":
            items = fn.load_tracks_from_artist(sp, source_id)
        elif source_type == "liked_songs":
            items = fn.load_my_saved_tracks(sp)
        elif source_type == "top_tracks":
            items = fn.load_my_top_tracks(sp)
        elif source_type == "recently_played":
            items = fn.load_my_recently_played(sp)
        else:
            raise ValueError(f"Unknown source_type: {source_type}")
        
        fn.df_to_duckdb(con, items, table_name)
    else:
        items = fn.duckdb_to_df(con, table_name)
    
    if verbose:
        display(f"{table_name}: {len(items)} records")
        display(items.head())

    return items

    

## geting basic data 

### my followed artist 

In [8]:


table_age = fn.duckdb_table_age(con, "followed_artist")

print(f"table age: {table_age} days old")


if table_age is None or table_age > 1.0:
    print("Refreshing followed_artist table...")

    followed_artist = fn.get_followed_artists_df(sp)
    display(followed_artist.head())

    print("Loading followed_artist table from DuckDB...")
    fn.df_to_duckdb(con, followed_artist, "followed_artist")
else:
    followed_artist = fn.duckdb_to_df(con, "followed_artist")
    print("Loaded followed_artist table from DuckDB.")


display(f"Followed Artists: {len(followed_artist)} records")
display(followed_artist.head())


table age: 0.3375005415164762 days old
Loaded followed_artist table from DuckDB.


'Followed Artists: 47 records'

,artist_id,name,followers,popularity,genres,uri
0,7I95CM75shzCjHuTzrepjM,Nova Twins,182010,52,,spotify:artist:7I95CM75shzCjHuTzrepjM
1,2qybVy8UqYXDx6x6BZedPE,Shadow Realm,239,10,,spotify:artist:2qybVy8UqYXDx6x6BZedPE
2,2mKV7sPvlBvzyPLeZhzT6q,Skyebrows,554,25,,spotify:artist:2mKV7sPvlBvzyPLeZhzT6q
3,2gTWLwanLiNZFR0iNPD847,EZ Band,119744,43,"latin country, tejano, norteño, cumbia norteña",spotify:artist:2gTWLwanLiNZFR0iNPD847
4,5qJEtz7aC2nwA9LsjfkGVM,Haku.,169079,44,japanese indie,spotify:artist:5qJEtz7aC2nwA9LsjfkGVM


### my liked songs

In [9]:
my_liked_songs = get_item("my_liked_songs", "liked_songs", None)


table age: 0.330912521749183 days old


'my_liked_songs: 1736 records'

,saved_at,track_id,track_name,album_name,album_id,artist_name,artist_id,duration_ms,explicit,popularity,is_local,uri
0,2025-12-07T14:37:58Z,69qPipyf6iRMPmnkBDGSLp,Thong Song (Motown Choir),Thong Song (Motown Choir),32wGCu2YCK4lfgdijS3lZB,"gino, 1950s Orchestra",3M22l9h7nL6oPStpf5AK6p,180961,True,33,False,spotify:track:69qPipyf6iRMPmnkBDGSLp
1,2025-12-07T14:34:30Z,4GgBpJYckh7S5Hr0OEmupd,Thong Song,Unleash The Dragon,77s7LHeb6Z7gDXWqCkyFrp,Sisqo,6x9QLdzo6eBZxJ1bHsDkjg,253733,False,50,False,spotify:track:4GgBpJYckh7S5Hr0OEmupd
2,2025-12-02T21:16:26Z,3WwoUlqRzqd3fAnMH8CXWJ,Joe's Rogaine Experience,Joe's Rogaine Experience,5vFGbEPRn8lsNLqpkvvuPP,Skyebrows,2mKV7sPvlBvzyPLeZhzT6q,264192,False,23,False,spotify:track:3WwoUlqRzqd3fAnMH8CXWJ
3,2025-11-29T00:42:11Z,7mykoq6R3BArsSpNDjFQTm,I Really Want to Stay at Your House,"Cyberpunk 2077: Radio, Vol. 2 (Original Soundt...",1VGVJdmvOSRK2w9RKXk18A,"Rosa Walton, Hallie Coggins",1X0HaTcdkHW7LviblBiEeq,246652,False,78,False,spotify:track:7mykoq6R3BArsSpNDjFQTm
4,2025-11-22T17:14:10Z,3jaZi4JMoh1o5znSKNqIgI,Again,Again,1WxeeVTaJxXhORenFx0Xbk,"Shadow Realm, Meg Wills",2qybVy8UqYXDx6x6BZedPE,249390,False,25,False,spotify:track:3jaZi4JMoh1o5znSKNqIgI


## new_liked_songs

In [10]:
new_liked_songs = duckdb.query(
"""
select * from my_liked_songs 
order by saved_at 
desc limit 100
"""
).to_df()

display(f"new_liked_songs: {len(new_liked_songs)}")
display(new_liked_songs.head())

# let's save this as a Playlist on Spotify
playlist_id = fn.create_playlist(
    sp,
    name="**New Liked Songs",
    description="My 100 most recently liked songs, updated via Spotify API",
    public=True,
    overwrite_if_exists=True
)

fn.add_tracks_to_playlist(sp, playlist_id, new_liked_songs["uri"].tolist())


'new_liked_songs: 100'

,saved_at,track_id,track_name,album_name,album_id,artist_name,artist_id,duration_ms,explicit,popularity,is_local,uri
0,2025-12-07T14:37:58Z,69qPipyf6iRMPmnkBDGSLp,Thong Song (Motown Choir),Thong Song (Motown Choir),32wGCu2YCK4lfgdijS3lZB,"gino, 1950s Orchestra",3M22l9h7nL6oPStpf5AK6p,180961,True,33,False,spotify:track:69qPipyf6iRMPmnkBDGSLp
1,2025-12-07T14:34:30Z,4GgBpJYckh7S5Hr0OEmupd,Thong Song,Unleash The Dragon,77s7LHeb6Z7gDXWqCkyFrp,Sisqo,6x9QLdzo6eBZxJ1bHsDkjg,253733,False,50,False,spotify:track:4GgBpJYckh7S5Hr0OEmupd
2,2025-12-02T21:16:26Z,3WwoUlqRzqd3fAnMH8CXWJ,Joe's Rogaine Experience,Joe's Rogaine Experience,5vFGbEPRn8lsNLqpkvvuPP,Skyebrows,2mKV7sPvlBvzyPLeZhzT6q,264192,False,23,False,spotify:track:3WwoUlqRzqd3fAnMH8CXWJ
3,2025-11-29T00:42:11Z,7mykoq6R3BArsSpNDjFQTm,I Really Want to Stay at Your House,"Cyberpunk 2077: Radio, Vol. 2 (Original Soundt...",1VGVJdmvOSRK2w9RKXk18A,"Rosa Walton, Hallie Coggins",1X0HaTcdkHW7LviblBiEeq,246652,False,78,False,spotify:track:7mykoq6R3BArsSpNDjFQTm
4,2025-11-22T17:14:10Z,3jaZi4JMoh1o5znSKNqIgI,Again,Again,1WxeeVTaJxXhORenFx0Xbk,"Shadow Realm, Meg Wills",2qybVy8UqYXDx6x6BZedPE,249390,False,25,False,spotify:track:3jaZi4JMoh1o5znSKNqIgI


## cream of crop

In [11]:
cream_of_crop = duckdb.query(
"""
select * from my_liked_songs 
order by popularity 
desc limit 100
"""
).to_df()

display(f"cream_of_crop: {len(cream_of_crop)}")
display(cream_of_crop.head())

# let's save this as a Playlist on Spotify
playlist_id = fn.create_playlist(
    sp,
    name="**Cream of Crop",
    description="My 100 most popular liked songs, updated via Spotify API",
    public=True,
    overwrite_if_exists=True
)

fn.add_tracks_to_playlist(sp, playlist_id, cream_of_crop["uri"].tolist())

'cream_of_crop: 100'

,saved_at,track_id,track_name,album_name,album_id,artist_name,artist_id,duration_ms,explicit,popularity,is_local,uri
0,2025-09-03T18:14:36Z,1CPZ5BxNNd0n0nF4Orb9JS,Golden,KPop Demon Hunters (Soundtrack from the Netfli...,14JkAa6IiFaOh5s0nMyMU9,"HUNTR/X, EJAE, AUDREY NUNA, REI AMI, KPop Demo...",2yNNYQBChuox9A5Ka93BIn,194607,False,98,False,spotify:track:1CPZ5BxNNd0n0nF4Orb9JS
1,2016-09-30T23:35:05Z,3ZOEytgrvLwQaqXreDs2Jx,Can't Stop,By the Way (Deluxe Edition),6deiaArbeoqp1xPEGdEKp1,Red Hot Chili Peppers,0L8ExT028jH3ddEcZwqJJ5,269000,False,87,False,spotify:track:3ZOEytgrvLwQaqXreDs2Jx
2,2015-07-20T21:23:34Z,57bgtoPSgt236HzfBOd8kj,Thunderstruck,The Razors Edge,4vu7F6h90Br1ZtYYaqfITy,AC/DC,711MCceyCBcFnzjGY4Q7Un,292333,False,87,False,spotify:track:57bgtoPSgt236HzfBOd8kj
3,2015-07-20T21:23:33Z,2zYzyRzz6pRmhPzyfMEC8s,Highway to Hell,Highway to Hell,10v912xgTZbjAtYfyKWJCS,AC/DC,711MCceyCBcFnzjGY4Q7Un,208110,False,87,False,spotify:track:2zYzyRzz6pRmhPzyfMEC8s
4,2019-09-21T18:43:08Z,6ECp64rv50XVz93WvxXMGF,This Love,Songs About Jane: 10th Anniversary Edition,5zClcGCSWj926AMjvBNSLc,Maroon 5,04gDigrS5kc9YWfZHwBETP,206200,False,87,False,spotify:track:6ECp64rv50XVz93WvxXMGF


## discover_these 

In [12]:
top_artist = followed_artist.head(10)['artist_id'].to_list()

disc_these = pd.DataFrame()
for artist_id in top_artist:
    artist_albums = fn.get_artist_top_tracks_df(sp, artist_id)
    disc_these = pd.concat([disc_these, artist_albums], ignore_index=True)

display(f"disc_these: {len(disc_these)}")
display(disc_these.head())

# let's save this as a Playlist on Spotify
playlist_id = fn.create_playlist(
    sp,
    name="**discover these",
    description="Best songs from recently followed artist, updated via Spotify API",
    public=True,
    overwrite_if_exists=True
)

fn.add_tracks_to_playlist(sp, playlist_id, disc_these["uri"].tolist())


'disc_these: 82'

,track_id,name,album_name,album_id,artist_name,artist_ids,popularity,duration_ms,explicit,preview_url,uri
0,6sIbv1oWOuma2qV907MUbk,1x1 (feat. Nova Twins),POST HUMAN: SURVIVAL HORROR,0e1WaSNDZnoPixaxDNdWo4,"Bring Me The Horizon, Nova Twins","1Ffb6ejR6Fe5IamqA5oRUF, 7I95CM75shzCjHuTzrepjM",67,209598,True,None,spotify:track:6sIbv1oWOuma2qV907MUbk
1,4WWogojK9PF7sbMvmqXGMV,Glory,Parasites & Butterflies,5uRq9tAqhsULMA35OL37Uk,Nova Twins,7I95CM75shzCjHuTzrepjM,46,222373,False,None,spotify:track:4WWogojK9PF7sbMvmqXGMV
2,3XRknZ0U5NKW8r5g44ptA6,Antagonist,Supernova,41ycYGcnhkDb3pFkL8vSPJ,Nova Twins,7I95CM75shzCjHuTzrepjM,44,186173,True,None,spotify:track:3XRknZ0U5NKW8r5g44ptA6
3,2tifAgNsyjjL20h1QKtVTe,Monsters,Parasites & Butterflies,5uRq9tAqhsULMA35OL37Uk,Nova Twins,7I95CM75shzCjHuTzrepjM,39,211786,False,None,spotify:track:2tifAgNsyjjL20h1QKtVTe
4,34yGnRiItM9kXHCXdYoQ2p,N.O.V.A,Parasites & Butterflies,5uRq9tAqhsULMA35OL37Uk,Nova Twins,7I95CM75shzCjHuTzrepjM,41,160960,True,None,spotify:track:34yGnRiItM9kXHCXdYoQ2p


## let's mix some playlists

### Covers++

In [13]:

# load some playlists
Covers = get_item("Covers", "playlist", "6jfY6NVENX592ZhLizN4HO",verbose=False)
AI_Covers = get_item("AI_Covers", "playlist", "5xooQuxBYK7ZXN4dhSQ9GL",verbose=False)
NTS_Covers = get_item("NTS_Covers", "playlist", "53pyL7jy1hbFbttiZZ8g1D",verbose=False)

# sort by popularity
Covers = Covers.sort_values(by=['popularity'], ascending=False).reset_index(drop=True)
AI_Covers = AI_Covers.sort_values(by=['popularity'], ascending=False).reset_index(drop=True)
NTS_Covers = NTS_Covers.sort_values(by=['popularity'], ascending=False).reset_index(drop=True)

# combine, drop duplicates, re-sort
CoversPP = pd.concat(
    [
        Covers.head(60), 
        AI_Covers.head(60),
        NTS_Covers.head(20)
    ], 
    ignore_index=True
    )
CoversPP = CoversPP.drop_duplicates()
CoversPP = CoversPP.sort_values(by=['popularity'], ascending=False).reset_index(drop=True)

display(f"CoversPP: {len(CoversPP)}")
display(CoversPP.head())

# save the playlist 
playlist_id = fn.create_playlist(
    sp,
    name="**Covers ++",
    description="Some of the Best Covers from my picks and AI, updated via Spotify API",
    public=True,
    overwrite_if_exists=True
)

fn.add_tracks_to_playlist(sp, playlist_id, CoversPP["uri"].tolist())


table age: 0.33085943421279945 days old
table age: 0.33082788658638795 days old
table age: 0.33080261819616513 days old


'CoversPP: 130'

,source_type,source_id,track_id,name,album_name,album_id,artist_name,artist_ids,popularity,duration_ms,explicit,preview_url,uri
0,6jfY6NVENX592ZhLizN4HO,playlist,1MTQHCpraD4S8g5PAFKzoj,Behind Blue Eyes,Results May Vary,3oUoQ0UH7Rv06x1kpSjS36,Limp Bizkit,165ZgPlLkK7bf5bDoFc6Sb,79,269973,False,<NA>,spotify:track:1MTQHCpraD4S8g5PAFKzoj
1,6jfY6NVENX592ZhLizN4HO,playlist,1OxcIUqVmVYxT6427tbhDW,Fly Away,5,6MCNMOCRsh6nxs7PNzc0zN,Lenny Kravitz,5gznATMVO85ZcLTkE9ULU7,77,221332,False,<NA>,spotify:track:1OxcIUqVmVYxT6427tbhDW
2,6jfY6NVENX592ZhLizN4HO,playlist,5z6xHjCZr7a7AIcy8sPBKy,Smooth Criminal,Anthology,5klPnHQ5dy6Qm2Ul7h1lp5,Alien Ant Farm,6TZdvF1kFzwnQLgHQynzsO,75,209266,False,<NA>,spotify:track:5z6xHjCZr7a7AIcy8sPBKy
3,6jfY6NVENX592ZhLizN4HO,playlist,7uSsHbBFFAnkRQR1rDwP3L,Black Betty - Single Edit,Tonight Alright,5WqHoLszhaCZHgtTebMx8X,Spiderbait,6P7kkhED6EPrfoZuxz20Fo,68,205973,False,<NA>,spotify:track:7uSsHbBFFAnkRQR1rDwP3L
4,6jfY6NVENX592ZhLizN4HO,playlist,5MYsvAONqcsIVOwu2HTQ3p,Faith,"Three Dollar Bill, Y'all $",3I34EprFC0rlAO4pKNeX1t,Limp Bizkit,165ZgPlLkK7bf5bDoFc6Sb,67,146480,True,<NA>,spotify:track:5MYsvAONqcsIVOwu2HTQ3p


#### this is just a more sql way of doing the same thing
* similar output

```python 


# load some playlists
Covers = get_item("Covers", "playlist", "6jfY6NVENX592ZhLizN4HO",verbose=False)
AI_Covers = get_item("AI_Covers", "playlist", "5xooQuxBYK7ZXN4dhSQ9GL",verbose=False)
NTS_Covers = get_item("NTS_Covers", "playlist", "53pyL7jy1hbFbttiZZ8g1D",verbose=False)


# display(NTS_Covers.head())

CoversPP = duckdb.query(
"""

select 
distinct * from 
(

    select * from 
    (
        select * from Covers
        order by popularity desc 
        limit 60
    ) A

    union all 

    select * from     
    (    
        select * from AI_Covers 
        order by popularity desc 
        limit 60
    ) B

    union all

    select * from     
    (    
        select * from NTS_Covers 
        order by popularity desc 
        limit 20
    ) C

) D
order by popularity 
desc limit 150

"""
).to_df()


display(f"CoversPP: {len(CoversPP)}")
display(CoversPP.head())

# save the playlist 
playlist_id = fn.create_playlist(
    sp,
    name="**Covers ++",
    description="Some of the Best Covers from my picks and AI, updated via Spotify API",
    public=True,
    overwrite_if_exists=True
)

fn.add_tracks_to_playlist(sp, playlist_id, CoversPP["uri"].tolist())
```